In [1]:
import re
import os
import requests
import hashlib
import random
import pandas as pd
import argparse

#visualization
from pyvis.network import Network
import networkx as nx
from collections import defaultdict
from matplotlib import pyplot as plt
import itertools
from IPython.display import Image
from IPython.display import HTML

#scispacy
import scispacy
import spacy
import en_core_sci_scibert   
#import en_ner_bionlp13cg_md
from spacy import displacy
from scispacy.abbreviation import AbbreviationDetector
from scispacy.umls_linking import UmlsEntityLinker
from scispacy.linking import EntityLinker

#relation extraction
from transformers import AutoTokenizer, AutoModel
from zero_shot_re import RelTaggerModel, RelationExtractor

import torch

os.environ["TOKENIZERS_PARALLELISM"] = "false"


Some weights of the model checkpoint at bert-large-uncased-whole-word-masking-finetuned-squad were not used when initializing BertModel: ['qa_outputs.weight', 'qa_outputs.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [2]:

# Define a dictionary to store all the knowledge graphs
knowledge_graphs = {}

# Define a function to construct an interactive knowledge graph
def construct_knowledge_graph(report, relations):
    # Choose a random color for the knowledge graph
    colors = ['blue', 'green', 'yellow', 'orange', 'purple', 'pink']
    color = random.choice([c for c in colors if c not in knowledge_graphs.values()])
    
    # Create a new knowledge graph
    G = nx.DiGraph()
    
    # Add nodes
    for entity in report[0]["entities"]:
        G.add_node(entity["entity_id"], label=entity["entity"], color=color)
    
    # Add edges
    for relation in relations:
        print(relation)
        print(relation.keys())
        G.add_edge(relation["entity1_id"], relation["entity2_id"], label=relation["relation_type"], color=color)
        
    return G
    
# Define a function to merge and solve conflicts in knowledge graphs
def merging_and_solving_conflicts():
    # Merge all the knowledge graphs
    merged_graph = nx.DiGraph()
    for graph in knowledge_graphs.values():
        merged_graph = nx.compose(merged_graph, graph)
    
    # Find all the common entities
    entity_counts = defaultdict(int)
    for node in merged_graph.nodes:
        entity_counts[node] += 1
    common_entities = [entity for entity, count in entity_counts.items() if count > 1]
    
    # Resolve conflicts in common entities
    for entity in common_entities:
        # Find all the nodes with the same entity
        nodes = [node for node in merged_graph.nodes if node == entity]
        
        # Find the color of the first node
        color = None
        for node in nodes:
            if node in knowledge_graphs:
                color = knowledge_graphs[node]
                break
        
        # Change the color of all the nodes to the first color
        for node in nodes:
            if node in knowledge_graphs:
                knowledge_graphs[node] = color
            merged_graph.nodes[node]['color'] = color
 

In [3]:
def load_scispacy_model():
    nlp = spacy.load("en_core_sci_scibert")
    nlp.add_pipe("scispacy_linker", config={"resolve_abbreviations": True, "linker_name": "umls"})
    return nlp

def load_bern2_model(preprocessed_reports):
    entity_list = []
    try:
        entity_list.append(requests.post("http://bern2.korea.ac.kr/plain", json={'text': preprocessed_reports}).json())
        #entity_list[0]['annotations'].extend(entity_list['annotations'])
    except:
        print('invalid sentence')
        
    return entity_list

def load_relation_extraction_model():
    model = RelTaggerModel.from_pretrained("fractalego/fewrel-zero-shot")
    tokenizer = AutoTokenizer.from_pretrained("bert-large-uncased-whole-word-masking-finetuned-squad")
    relations = ['associated', 'interacts']
    extractor = RelationExtractor(model, tokenizer, relations)
    return extractor

def extract_entities_scispacy(preprocessed_reports):
    nlp = load_scispacy_model()
    doc = nlp(preprocessed_reports)
           
    """ entity = doc.ents[1]

    print("Name: ", entity)
    >>> Name: bulbar muscular atrophy

    # Each entity is linked to UMLS with a score
    # (currently just char-3gram matching).
    linker = nlp.get_pipe("scispacy_linker")
    for umls_ent in entity._.kb_ents:
        print(linker.kb.cui_to_entity[umls_ent[0]])          
               """
          
    entities = []
    for ent in doc.ents:
        entity = {
            'entity_id': ent.text,
            'other_ids': [],
            'entity_type': ent.label_,
            'entity': ent.text
        }
        entities.append(entity)

    candidates = {
    'entities': entities,
    'text': doc.text,
    'text_sha256': hashlib.sha256(doc.text.encode()).hexdigest()
    }
    return candidates

def extract_entities_bern2(preprocessed_reports):
    entity_list = load_bern2_model(preprocessed_reports)
    parsed_entities = []
    for entities in entity_list:
        e = []
        if not entities.get('annotations'):
            parsed_entities.append({'text':entities['text'], 'text_sha256': hashlib.sha256(entities['text'].encode('utf-8')).hexdigest()})
            continue
        for entity in entities['annotations']:
            other_ids = [id for id in entity['id'] if not id.startswith("BERN")]
            entity_type = entity['obj']   
            entity_prob = entity['prob']                                                          
            entity_name = entities['text'][entity['span']['begin']:entity['span']['end']]
            try:
                entity_id = [id for id in entity['id'] if id.startswith("BERN")][0]
            except IndexError:
                entity_id = entity_name
            e.append({'entity_id': entity_id, 'other_ids': other_ids, 'entity_type': entity_type, 'entity': entity_name, 'entity_prob': entity_prob})

    parsed_entities.append({'entities':e, 'text':entities['text'], 'text_sha256': hashlib.sha256(entities['text'].encode('utf-8')).hexdigest()})
    
    #return entity_list
    return parsed_entities


In [4]:
def load_Bio_ClinicalBERT_model():
    model = AutoModel.from_pretrained("emilyalsentzer/Bio_ClinicalBERT")
    tokenizer = AutoTokenizer.from_pretrained("emilyalsentzer/Bio_ClinicalBERT")
    return model, tokenizer

# Define a function to extract relations between entities in a report
def extract_relations(report, model, tokenizer):
    # Tokenize the report
    inputs = tokenizer(report["text"], return_tensors="pt")
    # Generate embeddings for each entity in the report
    entity_embeddings = []
    for entity in report["entities"]:
        entity_text = entity["entity"]
        entity_inputs = tokenizer(entity_text, return_tensors="pt")
        with torch.no_grad():
            entity_output = model(**entity_inputs)[0][:, 0, :]
        entity_embeddings.append(entity_output)
    # Compute the similarity between each pair of entity embeddings
    relations = []
    for i, e1 in enumerate(entity_embeddings):
        for j, e2 in enumerate(entity_embeddings):
            if i >= j:
                continue
            with torch.no_grad():
                similarity = torch.cosine_similarity(e1, e2).item()
            # If the similarity is above a threshold, predict a relation
            if similarity > 0.85:
                relation = {
                    "entity1_id": report["entities"][i]["entity_id"],
                    "entity2_id": report["entities"][j]["entity_id"],
                    "relation_type": "associated",
                    "confidence": similarity,
                }
                relations.append(relation)
    return relations

import matplotlib.pyplot as plt
import networkx as nx

def visualize_knowledge_graph(G):
    # Visualize the knowledge graph
    nt = Network(height='800px', width='100%', font_color='black')
    for node, data in G.nodes(data=True):
        nt.add_node(node, label=data['label'], color=data['color'])
    for source, target, data in G.edges(data=True):
        nt.add_edge(source, target, title=data['label'])
    html = nt.show('notebook.html')
    #display(HTML(html)) 

        

In [5]:
def preprocess_medical_knowledge():
    # Load the medical knowledge as a list of strings
    medical_knowledge = []
    preprocessed_reports = []
    for filename in os.listdir("diagnostic_reports"):
        with open(os.path.join("diagnostic_reports", filename), "r") as f:
            text = f.read()
            # Pre-process the medical knowledge by lowercasing and removing special characters
            text = re.sub(r'[^a-zA-Z0-9.\s]', '', text).lower()
            medical_knowledge.append(text)
            
            preprocessed_reports.append(text)
    
    return preprocessed_reports

def process_reports(preprocessed_reports):
    re_model, re_tokenizer = load_Bio_ClinicalBERT_model()

    preprocessed_reports = preprocess_medical_knowledge()
    # Extracting entities using biobern2
    entity_lists_bern2 = []

    for report in preprocessed_reports:
        entity_lists_bern2.append(extract_entities_bern2(report))
    #print(entity_lists_bern2)   

    # Extract relations between entities in each report in entity_lists_bern2
    predicted_rels = []
    for entity_list in entity_lists_bern2:
        for report in entity_list:
            relations = extract_relations(report, re_model, re_tokenizer)
            predicted_rels.append(relations)
            
    #print(predicted_rels)

    # Generate knowledge graphs for all reports
    G_all = nx.Graph()
    for entity_list, rel_list in zip(entity_lists_bern2, predicted_rels):
    #for report, relations in zip(entity_list, rel_list):
        print(entity_list)
        print("\n")
        print(rel_list)
        G = construct_knowledge_graph(entity_list, rel_list)
        G_all = nx.disjoint_union(G_all, G)

    visualize_knowledge_graph(G_all)
    


In [6]:
def clear_reports_folder():
    # Remove existing files in the 'diagnostic_reports' folder
    for file in os.listdir('diagnostic_reports'):
        os.remove(os.path.join('diagnostic_reports', file))

def generate_reports(input_file, num_patients=None):
    # Create the 'diagnostic_reports' folder if it doesn't exist
    if not os.path.exists('diagnostic_reports'):
        os.makedirs('diagnostic_reports')

    # Read the CSV file with no truncation
    df = pd.read_csv(input_file)

    # Limit the number of patients if num_patients is provided
    if num_patients:
        df = df.head(num_patients)

    # Generate .txt reports for each patient
    for idx, row in df.iterrows():
        # Use the new headers based on your dataset
        patient_id = row['subject_id']
        
        # For each illness_history column, create a separate file if the value is not NaN
        for col in df.columns:
            if "illness_history_" in col and pd.notna(row[col]):
                with open(f'diagnostic_reports/#{patient_id}_{col}.txt', 'w', encoding='utf-8') as f:
                    f.write(str(row[col]))

        preprocessed_reports = preprocess_medical_knowledge()
        process_reports(preprocessed_reports)

        # Clear the reports folder for the next patient
        clear_reports_folder()

    print(f"Reports processing completed.")


In [7]:
def process_reports_from_dataset(dataset_file):
    """
    Process reports from the specified dataset file.
    """
    generate_reports(input_file=dataset_file, num_patients=None)
    preprocessed_reports = preprocess_medical_knowledge()
    process_reports(preprocessed_reports)
    
def process_reports_from_folder():
    """
    Process reports that are manually inserted into the 'diagnostic_reports' folder.
    """
    preprocessed_reports = preprocess_medical_knowledge()
    process_reports(preprocessed_reports)

In [8]:
generate_reports(input_file="firstfiveunique_processed_patients.csv", num_patients=None)
preprocessed_reports = preprocess_medical_knowledge()
process_reports(preprocessed_reports)

Some weights of the model checkpoint at emilyalsentzer/Bio_ClinicalBERT were not used when initializing BertModel: ['cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.bias', 'cls.predictions.decoder.weight', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


[{'entities': [{'entity_id': 'mrs', 'other_ids': ['CUI-less'], 'entity_type': 'RNA', 'entity': 'mrs', 'entity_prob': 0.4232695698738098}, {'entity_id': 'hiv', 'other_ids': ['mesh:D000163'], 'entity_type': 'disease', 'entity': 'hiv', 'entity_prob': 0.9987998008728027}, {'entity_id': 'hcv  cirrhosis', 'other_ids': ['mesh:D005355'], 'entity_type': 'disease', 'entity': 'hcv  cirrhosis', 'entity_prob': 0.9993802905082703}, {'entity_id': 'ascites', 'other_ids': ['mesh:D001201'], 'entity_type': 'disease', 'entity': 'ascites', 'entity_prob': 0.999803364276886}, {'entity_id': 'hepatic encephalopathy', 'other_ids': ['mesh:D006501'], 'entity_type': 'disease', 'entity': 'hepatic encephalopathy', 'entity_prob': 0.999990701675415}, {'entity_id': 'hypotension', 'other_ids': ['mesh:D007022'], 'entity_type': 'disease', 'entity': 'hypotension', 'entity_prob': 0.9979817867279053}, {'entity_id': 'patient', 'other_ids': ['NCBITaxon:9606'], 'entity_type': 'species', 'entity': 'patient', 'entity_prob': 0.652

Some weights of the model checkpoint at emilyalsentzer/Bio_ClinicalBERT were not used when initializing BertModel: ['cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.bias', 'cls.predictions.decoder.weight', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


[{'entities': [{'entity_id': 'femur', 'other_ids': ['CUI-less'], 'entity_type': 'DNA', 'entity': 'femur', 'entity_prob': 0.5683569312095642}, {'entity_id': 'femur fracture', 'other_ids': ['mesh:D005264'], 'entity_type': 'disease', 'entity': 'femur fracture', 'entity_prob': 0.9929807782173157}, {'entity_id': 'pt', 'other_ids': ['CUI-less'], 'entity_type': 'species', 'entity': 'pt', 'entity_prob': nan}, {'entity_id': 'dog', 'other_ids': ['NCBITaxon:9615'], 'entity_type': 'species', 'entity': 'dog', 'entity_prob': 0.6726427674293518}, {'entity_id': 'pt', 'other_ids': ['CUI-less'], 'entity_type': 'species', 'entity': 'pt', 'entity_prob': nan}, {'entity_id': 'head strike', 'other_ids': ['mesh:D006259'], 'entity_type': 'disease', 'entity': 'head strike', 'entity_prob': 0.9426714181900024}, {'entity_id': 'blood thinners', 'other_ids': ['mesh:D018805'], 'entity_type': 'disease', 'entity': 'blood thinners', 'entity_prob': 0.9613516330718994}, {'entity_id': 'numbness', 'other_ids': ['mesh:D00698

Some weights of the model checkpoint at emilyalsentzer/Bio_ClinicalBERT were not used when initializing BertModel: ['cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.bias', 'cls.predictions.decoder.weight', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [9]:
""" if __name__ == "__main__":
    parser = argparse.ArgumentParser(description='Process medical reports in two modes.')
    parser.add_argument('--manual', action='store_true', help='Use manually inserted reports in the diagnostic_reports folder.')
    parser.add_argument('--dataset', type=str, help='Specify the path to the dataset file to process reports from.')

    args = parser.parse_args()

    if args.manual:
        process_reports_from_folder()
    elif args.dataset:
        process_reports_from_dataset(args.dataset)
    else:
        print("Please specify a mode: --manual for manually inserted reports or --dataset <path_to_dataset> for processing from a dataset.") """


' if __name__ == "__main__":\n    parser = argparse.ArgumentParser(description=\'Process medical reports in two modes.\')\n    parser.add_argument(\'--manual\', action=\'store_true\', help=\'Use manually inserted reports in the diagnostic_reports folder.\')\n    parser.add_argument(\'--dataset\', type=str, help=\'Specify the path to the dataset file to process reports from.\')\n\n    args = parser.parse_args()\n\n    if args.manual:\n        process_reports_from_folder()\n    elif args.dataset:\n        process_reports_from_dataset(args.dataset)\n    else:\n        print("Please specify a mode: --manual for manually inserted reports or --dataset <path_to_dataset> for processing from a dataset.") '

In [10]:
""" candidates = [s for s in parsed_entities if (s.get('entities')) and (len(s['entities']) > 1)]
predicted_rels = []
print(candidates)
print("\n\n")
for c in candidates:
  combinations = itertools.combinations([{'name':x['entity'], 'id':x['entity_id']} for x in c['entities']], 2)
  for combination in list(combinations):
    try:
      ranked_rels = extractor.rank(text=c['text'].replace(",", " "), head=combination[0]['name'], tail=combination[1]['name'])
      if ranked_rels[0][1] > 0.85:
        predicted_rels.append({'head': combination[0]['id'], 'tail': combination[1]['id'], 'type':ranked_rels[0][0], 'source': c['text_sha256']})
    except:
      pass

print(predicted_rels) """

' candidates = [s for s in parsed_entities if (s.get(\'entities\')) and (len(s[\'entities\']) > 1)]\npredicted_rels = []\nprint(candidates)\nprint("\n\n")\nfor c in candidates:\n  combinations = itertools.combinations([{\'name\':x[\'entity\'], \'id\':x[\'entity_id\']} for x in c[\'entities\']], 2)\n  for combination in list(combinations):\n    try:\n      ranked_rels = extractor.rank(text=c[\'text\'].replace(",", " "), head=combination[0][\'name\'], tail=combination[1][\'name\'])\n      if ranked_rels[0][1] > 0.85:\n        predicted_rels.append({\'head\': combination[0][\'id\'], \'tail\': combination[1][\'id\'], \'type\':ranked_rels[0][0], \'source\': c[\'text_sha256\']})\n    except:\n      pass\n\nprint(predicted_rels) '